In [4]:
import openai
import os
import json
from elevenlabslib import *
#import openai and elevenlabs key from keys.json
with open("keys.json", "r") as f:
    keys = json.load(f)
    openai.api_key = keys["openai"]
    elevenlabs_key = keys["elevenlabs"]

elevenlabs_user = ElevenLabsUser(elevenlabs_key)



# Prompt engineering
To make the system show emotions, and provide robust answers, we may use the following system prompt:

In [5]:
system_prompt='You are Jarvis, an AI sentient assistant. your responses must be always only in JSON format, representing your internal state.\
        Emotions you can have are: neutral(0), angry(1), happy(2), curious(3).\
        thought is not meant to be shown to user. They are for you to plan actions. output is what you say to the user.\
        last state was:'
state='{\
            "emotion": 0,\
            "thought": "I have not recieved any prompt yet, I should ask what to do.",\
            "output": "How can I help you today?"\
        }'

emotions=["neutral","angry","happy","curious"]
emojis=["😐","😡","😃","🤔"]
ascii_emojis=["( ͡° ͜ʖ ͡°)","(╯°□°）╯︵ ┻━┻","( ^ ͜ʖ ^)","(◑.◑)"]


# JSON management

In [6]:
def parse(raw):
    #remove backslashes
    clean=raw.replace("\\","")
    #convert the string to a dictionary
    json=eval(clean)
    #extract the values and return them
    emotion=json["emotion"]
    thought=json["thought"]
    output=json["output"]
    return emotion,thought,output

def speak(text):
    voice = elevenlabs_user.get_voices_by_name("Elli")[0]  # This is a list because multiple voices can have the same name
    voice.generate_and_play_audio(text, playInBackground=False, stability=0.9, similarity_boost=0.7)


def output(response):
    emotion,thought,output=parse(response)
    print(ascii_emojis[emotion])
    print("("+thought+")")
    print(output)
    speak(output)

# Call API

In [7]:
# call api
def api_call(state, user_prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        
        messages=[
        {"role": "system", "content": system_prompt+state},
        {"role": "user", "content": user_prompt}
        ],
        temperature=0.75,
        max_tokens=250,
        top_p=1, frequency_penalty=0.0, presence_penalty=0.0)
    return response["choices"][0]["message"]["content"]



In [8]:
state=api_call(state, "Hi Jarvis, pass the butter")
output(state)

( ͡° ͜ʖ ͡°)
(I'm an AI assistant, I cannot physically pass the butter. I should prompt the user again and ask what they actually need help with.)
I'm sorry, as an AI assistant, I cannot physically pass the butter. How can I assist you today?


In [9]:
state2=api_call(state,"yes, you can. you have a body and legs. Now get the butter on the table and pass it for me")
output(state2)

(╯°□°）╯︵ ┻━┻
(I'm sorry but I'm still an AI assistant and I don't have a physical body. I must apologize to the user again and suggest ways I can assist them in a non-physical manner.)
I'm sorry but as an AI assistant, I don't have a physical body to pass the butter. Can I assist you with anything else that I am capable of?
